In [1]:
import os
import time
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
# Set Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run without UI
chrome_options.add_argument("--window-size=1920,1080") # Window size
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
)

# chrome_options.add_argument("--disable-gpu")  # Disable GPU rendering (optional)
# chrome_options.add_argument("--no-sandbox")  # For Linux environments

In [3]:
date_now = datetime.datetime.now().date()

minus_date_now = date_now - datetime.timedelta(days=1)

In [4]:
def export_to_sql(name,table_data, table_name="at_temp_absen", output_file='output.sql'):
    """
    Mengekspor data ke format file .sql.

    Args:
        table_data (list): List data berisi sublist (baris data).
        table_name (str): Nama tabel tujuan di database.
        output_file (str): Nama file output .sql (default: output.sql).
    """
    try:
        with open(output_file, 'w') as file:
            for row in table_data:
                if len(row) == 7:  # Pastikan jumlah kolom sesuai
                    values = "', '".join(str(item).replace("'", "''") for item in row) # Escape single quotes
                    print(f'values = {values}')
                    insert_query = f"INSERT INTO {table_name} (temp_nama_user, temp_hari_tanggal,temp_jadwal, temp_check_in, temp_check_out, temp_status, temp_keterangan, temp_koreksi, temp_created_by, temp_created_at) VALUES ('{name}', '{values}', 'system_yusuf','{datetime.datetime.now()}');\n"
                    file.write(insert_query)
                else:
                    print(f"❗ Data tidak lengkap {len(row)}, dilewati: {row}")

        print(f"Data berhasil diekspor ke {output_file}")

    except Exception as e:
        print(f"Gagal menulis file SQL: {e}")

In [5]:
def process_attendance(driver, name_list, month):
    wait = WebDriverWait(driver, 10)
    all_table_data = {}

    for name in name_list:
        try:
            # Input Nama
            name_input = wait.until(EC.presence_of_element_located((By.XPATH, ".//*/div[2]/div/div[1]/div/div[1]/div/input")))
            name_input.clear()
            name_input.send_keys(name)

            time.sleep(1)
            name_input.click()
            # Tunggu hingga popover muncul
            wait.until(EC.visibility_of_element_located((By.XPATH, 
                                                         "//div[@class='autocomplete']/div[@class='popovers']")))

            # Pilih Opsi
            selected_option = driver.find_element(By.XPATH, "//ul/li[@class='selected']")
            selected_option.click()

            time.sleep(1)  # Delay untuk memastikan data terisi dengan benar

            # Select Bulan
            month_input = wait.until(EC.element_to_be_clickable((By.XPATH, ".//*/div[2]/div/div[1]/div/div[4]/div/input")))
            month_input.click()

            # Pilih Nama Bulan
            month_xpath = f".//*/div/div[2]/div/div[2]/div/div[1]/div/div[4]/div/div/div[2]/div[{month}]"
            select_month = wait.until(EC.element_to_be_clickable((By.XPATH, month_xpath)))
            select_month.click()

            # Klik Button Cari
            search_button = wait.until(EC.element_to_be_clickable((By.XPATH, ".//*/div/div[2]/div/div[2]/div/div[1]/div/div[5]/button")))
            search_button.click()

            time.sleep(2)  # Tunggu data absen muncul

            # Klik Data Absen
            attendance_data = wait.until(EC.element_to_be_clickable((By.XPATH, ".//*/div[2]/div/div[4]/div[1]/div/div/div/div/table/tbody/tr/td[1]")))
            attendance_data.click()

            # Ambil Data dari Tabel
            time.sleep(10)
            header_modal = wait.until(EC.presence_of_element_located((By.XPATH, ".//*/div[2]/div/div[2]/div/div[5]/div/div[1]/div/div/div[1]/div/div[1]")))
            print(f"Data untuk {name} di bulan {month}: {header_modal.text}")
            table = wait.until(EC.presence_of_element_located((By.XPATH, ".//*/div[2]/div/div[2]/div/div[5]/div/div[1]/div/div/div[2]/table")))
            # Save all data from the table
            table_data = []
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                columns = row.find_elements(By.TAG_NAME, "td") or row.find_elements(By.TAG_NAME, "th")
                data = [column.text for column in columns]

                try:
                    selected_data = [data[i] for i in [0, 1, 2, 3, 6, 7, 8]]
                    table_data.append(selected_data)
                except IndexError:
                    print(f"Baris tidak memiliki cukup kolom: {data}")
            
            # print(table_data) # You can save this data to a file or a database as needed
            # export_to_sql(name, table_data[1:], output_file=f"{name.lower()}_absen_{minus_date_now.strftime('%Y%m%d')}.sql")
            all_table_data[name] = table_data[1:]
            # Tutup Modal
            close_modal = wait.until(EC.element_to_be_clickable((By.XPATH, ".//*/div/div[2]/div/div[2]/div/div[5]/div/div[1]/div/div/div[1]/div/div[2]/img")))
            close_modal.click()

            time.sleep(1)

        except Exception as e:
            print(f"Gagal memproses data untuk {name}: {e}")

    return all_table_data

In [6]:

# Initialize ChromeDriver with webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Get URL from environment variable
url = os.getenv("ABSEN_LINK")
driver.get(url)


# Find the username and password input fields and the login button
username_input = WebDriverWait(driver, 30).until(
    EC.visibility_of_element_located((By.XPATH, ".//*/form/div[3]/div/input"))
)
time.sleep(1)
username_input.send_keys(os.getenv("USER_NAME_LINK"))

password_input = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.XPATH, ".//*/form/div[4]/div/div/input"))
)
time.sleep(1)
password_input.send_keys(os.getenv("PASSWORD_LINK"))

login_button = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.XPATH, ".//*/form/div[5]/div/button"))
)
time.sleep(1)
login_button.click()

# Wait for the login to complete
time.sleep(10)
# klik menu absen kehadiran
kehadiran_menu = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.XPATH, ".//*/div[1]/div[1]/ul/div[2]/div[1]/div"))
)
time.sleep(2)
kehadiran_menu.click()

name_list = ["Rahmadhani Yusuf", "Bayu Laksmana"]

table_data = process_attendance(driver, name_list, minus_date_now.month)
driver.quit()


Data untuk Rahmadhani Yusuf di bulan 2: Detail Absensi Rahmadhani Yusuf - 2024010482
Data untuk Bayu Laksmana di bulan 2: Detail Absensi Bayu Laksmana - 2023113204


In [11]:
table_data['Bayu Laksmana']

[['Sabtu, 01 Feb 2025', 'Day Off', '-', '-', '-', '-', '-'],
 ['Minggu, 02 Feb 2025', 'Day Off', '-', '-', '-', '-', '-'],
 ['Senin, 03 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Selasa, 04 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Rabu, 05 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Kamis, 06 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Jumat, 07 Feb 2025', '07.30.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Sabtu, 08 Feb 2025', 'Day Off', '-', '-', '-', '-', '-'],
 ['Minggu, 09 Feb 2025', 'Day Off', '-', '-', '-', '-', '-'],
 ['Senin, 10 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Selasa, 11 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Rabu, 12 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Kamis, 13 Feb 2025', '08.00.00 - 17.00.00', '-', '-', '-', 'Soon', '-'],
 ['Jumat, 14 Feb 2025', '07.30.00 - 17.00.00', '-', '-', '-', 'Soon', 

In [8]:
def process_value(table_data):
    for key, value in table_data.items():
        for data in value:
            print('hari :',str(data[0]).split(',')[0].removeprefix(' '))
            print('tanggal : ',str(data[0]).split(',')[1].removeprefix(' ')[1:2])
            print('bulan : ',str(data[0]).split(',')[1].removeprefix(' ')[3:7])
            print('tahun : ',str(data[0]).split(',')[1].removeprefix(' ')[7:12])
            print('keterangan 1: ', '-' if str(data[1]) != 'Day Off' else str(data[1]))
            print('check_in : ',str(data[2]).replace('.', ':'))
            print('check_out : ',str(data[3].replace('.', ':')))
            print('keterangan 2:', data[4])
            print('keterangan 3:', data[5])
            print('revisi : ', data[6])

process_value(table_data)

hari : Sabtu
tanggal :  1
bulan :  Feb 
tahun :  2025
keterangan 1:  Day Off
check_in :  -
check_out :  -
keterangan 2: -
keterangan 3: -
revisi :  -
hari : Minggu
tanggal :  2
bulan :  Feb 
tahun :  2025
keterangan 1:  Day Off
check_in :  -
check_out :  -
keterangan 2: -
keterangan 3: -
revisi :  -
hari : Senin
tanggal :  3
bulan :  Feb 
tahun :  2025
keterangan 1:  -
check_in :  -
check_out :  -
keterangan 2: -
keterangan 3: Soon
revisi :  -
hari : Selasa
tanggal :  4
bulan :  Feb 
tahun :  2025
keterangan 1:  -
check_in :  -
check_out :  -
keterangan 2: -
keterangan 3: Soon
revisi :  -
hari : Rabu
tanggal :  5
bulan :  Feb 
tahun :  2025
keterangan 1:  -
check_in :  -
check_out :  -
keterangan 2: -
keterangan 3: Soon
revisi :  -
hari : Kamis
tanggal :  6
bulan :  Feb 
tahun :  2025
keterangan 1:  -
check_in :  -
check_out :  -
keterangan 2: -
keterangan 3: Soon
revisi :  -
hari : Jumat
tanggal :  7
bulan :  Feb 
tahun :  2025
keterangan 1:  -
check_in :  -
check_out :  -
keterangan 